In [2]:
import plotly
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import scipy.stats as stats
import simpleaudio as sa

from plotly.subplots import make_subplots

empty = np.empty(0, dtype=int)

In [10]:
startfile  = 20
lastfile   = 20
fileprefix = 'data'
savefile   = True
savefileStr = '90%CI'

files = range(startfile,lastfile+1)
wave_obj = sa.WaveObject.from_wave_file("sound.wav") # sound that is played at end of calculations

runs = [1,2]
#data = pd.read_csv('./ModelData/'+fileprefix+str(file)+'.csv')

columnnames = ['Healthy', 'Infected', 'Cured', 'Dead']         # Choose from Healthy, Infected, Cured, Dead ['Infected']#
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]                      # Choose 5 Quantiles
#quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]                   # Choose 5 Quantiles
percentages = [0.15, 0.50]# top always included

var1 = 'V'
var2 = 'H'
R = ' - R'
Overview = pd.DataFrame()
i = 1
index = 0
quantilesrange = range(1,len(quantiles)+1)

#---------------------------------------#
for file in files: 
    data = pd.read_csv('./ModelData/'+fileprefix+str(file)+'.csv')
    simulations = len(data['Day'][(data['Day'] == 1)])
    
    #-------Moments-------START----------------------#
    grouped      = data.groupby(['Day']).quantile(0.5)
    maxinfected1 = grouped.max()['Infected1']
    topinfected1 = grouped.idxmax()['Infected1']
    interestdays = empty

    df1 = data[data['Day'] < topinfected1]
    df2 = data[data['Day'] > topinfected1]
    beforetop = df1.groupby(['Day']).quantile(0.5)
    aftertop = df2.groupby(['Day']).quantile(0.5)
    moments = [beforetop, aftertop]

    for moment in moments:
        for p in percentages:
            foundday = find_neighbours(maxinfected1*p)
            interestdays = np.append(interestdays,foundday)

    interestdays = np.append(interestdays,topinfected1)    
    allpercentages = np.append(percentages, [1, percentages[1], percentages[0]])
    interests = len(interestdays)    
    interestdays.sort()
    #-------Moments------END--------------------------#
    
    for nd in columnnames:        
        for day, daycount in zip(interestdays, range(0,interests)):
            index = index + 1
            Overview.loc[index,'File'] = int(file)
            Overview.loc[index,'Column'] = nd
            Overview.loc[index,'Day'] = day
            Overview.loc[index,'%'] = allpercentages[daycount]                    
            
            for run in (runs):
                founddays = empty                
                #-----Vertical Change----------START-------#
                for q, qcount in zip(quantiles, range(1,len(quantiles)+1)):
                    qdata = data.groupby(['Day']).quantile(q)[nd+str(run)].iloc[day]
                    if qcount == 3: main = qdata
                    Overview.loc[index,var1+str(qcount)+R+str(run)] = round(qdata,1)                
                #-----Vertical Change------------END-------# 
                
                #-----Horizontal Change---------START------#                
                if nd == 'Infected':                    
                    if daycount < 2:
                        for q, qcount in zip(quantiles, quantilesrange):
                            moment = data[data['Day'] < topinfected1].groupby(['Day']).quantile(q)
                            foundday = find_neighbours(main)
                            Overview.loc[index,var2+str(qcount)+R+str(run)] = foundday
                        
                    elif daycount > 2:
                        for q, qcount in zip(quantiles, quantilesrange):
                            moment = data[data['Day'] > topinfected1].groupby(['Day']).quantile(q)
                            foundday = find_neighbours(main)
                            Overview.loc[index,var2+str(qcount)+R+str(run)] = foundday
                        
                    elif daycount == 2:
                        for q, v in zip([quantiles[4], quantiles[3], quantiles[2]], [2, 2, 1]):
                            moment = data.groupby(['Day']).quantile(q)
                            foundday = find_neighbours(main)
                            founddays = np.append(founddays,foundday)
                        founddays.sort()
                        for qcount in (quantilesrange):
                            Overview.loc[index,var2+str(qcount)+R+str(run)] = founddays[qcount-1]                                                
                else:
                    for q, qcount in zip(quantiles, quantilesrange):
                        moment = data.groupby(['Day']).quantile(q)
                        foundday = find_neighbours(main)
                        Overview.loc[index,var2+str(qcount)+R+str(run)] = foundday                                
                #-------Horizontal Change--------END-------#
                
            varrun1 = data.groupby(['Day']).var()[nd+'1'].iloc[day]
            varrun2 = data.groupby(['Day']).var()[nd+'2'].iloc[day]
            addvar = np.round(np.divide(np.subtract(varrun1,varrun2),varrun1),2)            
            
            Overview.loc[index,'Quantiles'] = ', '.join(map(str, quantiles))
            Overview.loc[index,'Add.Var.'] = addvar               
            #Overview.loc[index,'Sims'] = round(simulations)  

if savefile == True:
    if startfile == lastfile:
        Overview.to_csv('./ModelProcessedData/Data'+str(startfile)+'.'+savefileStr+'.DataFrame.csv', index=False)
    else: 
        Overview.to_csv('./ModelProcessedData/Data'+str(startfile)+'-'+str(stopfile)+'.'+savefileStr+'.DataFrame.csv', index=False)
    

play_obj = wave_obj.play()
#play_obj.wait_done()

Overview

IndexError: index 3 is out of bounds for axis 0 with size 3

In [9]:
def find_neighbours(value):
  exactmatch=moment[moment['Infected'+str(i)]==value]
  if not exactmatch.empty:
      return exactmatch.index
  else:
      lowerneighbour_ind = moment[moment['Infected'+str(i)]<value]['Infected'+str(i)].idxmax()
      if v == 2: upperneighbour_ind = moment[moment['Infected'+str(i)]>value]['Infected'+str(i)].idxmin()
      return [lowerneighbour_ind]#, upperneighbour_ind]